# Compare-6: data in different timespan should include similar ICD code sets (ICD10CM/ICD10PCS)

Description: check if data in different timespan include similar ICD code sets (ICD10CM).

Starting Author: Amy Jin (amy@careset.com)

Date: July 27th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name1:                database 1 name in server
# 2）table_name1:             table1 name
# 3) col_name1:               column in table 1 to test
# 4) key1_1:                  key in table 1

# 5) db_name2:                database 2 name in server
# 6）table_name2:             table 2 name
# 7) col_name2:               column in table 2 to test
# 8) key2_1:                  key in table 2
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result: 
#       - distinct ICD count in table 1
#       - distinct ICD count in table 2
#       - distinct ICD count in both table 1 and table 2


def compare_6(db_name1, table_name1, col_name1, \
              key1_1, \
              db_name2, table_name2, col_name2, \
              key2_1):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL query to compute distinct ICD count in both table 1 and table 2
        query = ('''
            SELECT COUNT(DISTINCT table1.{col1})
            FROM {db1}.{t1} AS table1
            JOIN {db2}.{t2} AS table2
            ON table1.{key1_1} = table2.{key2_1} ;
        '''.format(db1 = db_name1, t1 = table_name1, col1 = col_name1, \
                   key1_1 = key1_1,  \
                  db2 = db_name2, t2 = table_name2, col2 = col_name2, \
                  key2_1 = key2_1 ))

        cur.execute(query)
    
        print ('Count of distinct {} in both {}.{} and {}.{} is:'.format(col_name1, db_name1, table_name1, \
                                                                         db_name2, table_name2))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            
        # MySQL query to compute distinct ICD count in table 1
        query = ('''
            SELECT COUNT(DISTINCT {col1})
            FROM {db}.{t1};
        '''.format(db = db_name1, t1 = table_name1, col1 = col_name1))

        cur.execute(query)
    
        print ('Count of distinct {} in {}.{} is:'.format(col_name1, db_name1, table_name1))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))

        # MySQL query to compute distinct ICD count in table 2
        query = ('''
            SELECT COUNT(DISTINCT {col1})
            FROM {db}.{t1};
        '''.format(db = db_name2, t1 = table_name2, col1 = col_name2))

        cur.execute(query)
    
        print ('Count of distinct {} in {}.{} is:'.format(col_name2, db_name2, table_name2))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            
        cur.close()
        connection.close()

## Test Example

In [3]:
compare_6('_amy', 'test_data_good','icd10cm', \
          'icd10cm', \
         '_amy', 'test_data_bad1','icd_dgns_cd',\
         'icd_dgns_cd')

Count of distinct icd10cm in both _amy.test_data_good and _amy.test_data_bad1 is:
0
Count of distinct icd10cm in _amy.test_data_good is:
2000
Count of distinct icd_dgns_cd in _amy.test_data_bad1 is:
138


## Internal Data Test Example

In [8]:
compare_6('icd_national', 'icd_national_RQ16', 'icd_dgns_cd', \
          'icd_dgns_cd',\
          'icd_national', 'icd_national_merged_RQ17', 'icd_dgns_cd', \
          'icd_dgns_cd')

Count of distinct icd_dgns_cd in both icd_national.icd_national_RQ16 and icd_national.icd_national_merged_RQ17 is:
59661
Count of distinct icd_dgns_cd in icd_national.icd_national_RQ16 is:
65645
Count of distinct icd_dgns_cd in icd_national.icd_national_merged_RQ17 is:
64818
